In [16]:
import pymysql
import pandas as pd

db = pymysql.connect(host = 'localhost',user = 'root',password = '123456',db = 'zeddhzm',charset='utf8')   
cur = db.cursor()   
sql = "SELECT * FROM jdcomment_new_2_classified"
cur.execute(sql)
result = cur.fetchall()
df = pd.DataFrame(list(result),columns=['ID','NICKNAME','CONTENT','CONTENT_LENGTH','CREATIONTIME','SCORE','REPLYCOUNT','IMAGESTATUS','IMAGECOUNT','USEFULVOTECOUNT','date','date_now','time_delta','COMPLETENESS','whether_useful'])
cur.close()
db.close()

dff = df.copy()

## 主观性计算规则
1. 文本分句
2. 计算每个句子的主观性
3. 每个句子主观性计算方式:  主观线索词*主观线索词权重，做加权累加和平均
4. 每个句子主观性求平均
5. 输出文本主观性

https://github.com/liuhuanyong/ZhuguanDetection

In [24]:
import jieba.posseg as pseg
import os
import re

class ZhuguanDetect:
    def __init__(self):
        #curdir = '/'.join(os.path.abspath(__file__).split('/')[:-1])
        curdir = 'C:/Users/Zedd/Desktop/'
        degree_file = os.path.join(curdir, 'dict/degree_words.txt')
        deny_file = os.path.join(curdir, 'dict/deny_words.txt')
        qingtai_file = os.path.join(curdir, 'dict/qingtai_words.txt')
        zhuzhang_file = os.path.join(curdir, 'dict/zhuzhang_words.txt')
        senti_file = os.path.join(curdir, 'dict/senti_words.txt')
        pingjia_file = os.path.join(curdir, 'dict/pingjia_words.txt')
        rencheng_file = os.path.join(curdir, 'dict/rencheng_words.txt')
        zhishi_file = os.path.join(curdir, 'dict/rencheng_words.txt')
        yiwen_file = os.path.join(curdir, 'dict/yiwen_words.txt')
        lianci_file = os.path.join(curdir, 'dict/lianci_words.txt')
        tanci_file = os.path.join(curdir, 'dict/tanci_words.txt')
        yuqi_file = os.path.join(curdir, 'dict/yuqi_words.txt')
        zhuangtai_file = os.path.join(curdir, 'dict/zhuangtai_words.txt')
        nengyuan_file = os.path.join(curdir, 'dict/nengyuan_words.txt')

        self.degree_words = self.load_words(degree_file)
        self.deny_words = self.load_words(deny_file)
        self.qingtai_words = self.load_words(qingtai_file)
        self.zhuzhang_words = self.load_words(zhuzhang_file)
        self.senti_words = self.load_words(senti_file)
        self.pingjia_words = self.load_words(pingjia_file)
        self.rencheng_words = self.load_words(rencheng_file)
        self.zhishi_words = self.load_words(zhishi_file)
        self.yiwen_words = self.load_words(yiwen_file)
        self.lianci_words = self.load_words(lianci_file)
        self.tanci_words = self.load_words(tanci_file)
        self.yuqi_words = self.load_words(yuqi_file)
        self.nengyuan_words = self.load_words(nengyuan_file)
        self.zhuangtai_words = self.load_words(zhuangtai_file)

    '''构造词典'''
    def load_words(self, file):
        return set([i.strip() for i in open(file, encoding='utf-8') if i.strip()])

    '''基于主观线索进行主观性分析'''
    def zhuguan(self, sent):
        scores = []
        segs = [[w.word, w.flag] for w in pseg.cut(sent)]
        for index, seg in enumerate(segs):
            wd = seg[0]
            postags = seg[1]
            score = self.score_words(wd)
            scores.append(score)
        return sum(scores)/len(segs)

    '''通过对主观性词语进行加权，用来计算'''
    def score_words(self, word):
        score = 0.0
        if word in self.degree_words:
            score = 0.75
        elif word in self.deny_words:
            score = 0.51
        elif word in self.qingtai_words:
            score = 0.81
        elif word in self.rencheng_words:
            score = 0.95
        elif word in self.zhuzhang_words:
            score = 0.98
        elif word in self.senti_words:
            score = 0.4
        elif word in self.pingjia_words:
            score = 0.95
        elif word in self.zhishi_words:
            score = 0.75
        elif word in self.yiwen_words:
            score = 0.9
        elif word in self.lianci_words:
            score = 0.88
        elif word in self.tanci_words:
            score = 0.75
        elif word in self.yuqi_words:
            score = 0.75
        elif word in self.nengyuan_words:
            score = 0.75
        elif word in self.zhuangtai_words:
            score = 0.6
        return score

    '''文章分句处理, 切分长句，冒号，分号，感叹号等做维护标识'''
    def split_sents(self, content):
        return [sentence for sentence in re.split(r'[？?！!。；;：:\n\r]', content) if sentence]

    '''程序主函数'''
    def detect(self, content):
        sents = self.split_sents(content)
        scores = []
        for sent in sents:
            sent_score = self.zhuguan(sent)
            scores.append(sent_score)
        return sum(scores)/len(scores)

    
'''调用class提供主观性提取接口'''    
def subjectivity_extract(sentence):
    handler = ZhuguanDetect()
    score = handler.detect(sentence)
    return score

In [25]:
subjectivity_extract('我喜欢你')

0.7666666666666666

In [20]:
dff2['subjectivity'] = dff2.CONTENT.apply(subjectivity_extract)

D:\Softwares Installed\Anaconda\anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [21]:
dff2

,ID,NICKNAME,CONTENT,CONTENT_LENGTH,CREATIONTIME,SCORE,REPLYCOUNT,IMAGESTATUS,IMAGECOUNT,USEFULVOTECOUNT,date,date_now,time_delta,COMPLETENESS,whether_useful,subjectivity
0,13872879901,j***5,机器刚到手，就发布降价信息！服了，气管接口粗糙，屏幕连个膜都没有！写的记忆14次，实际10次...,56,2020-03-04 18:21:47,1,14,1,2,47,2020-03-04,2021-01-24,326,4,1,0.212083
1,14916170593,海***行,你们这是@，知道么。用你们的血压计天天测都是120-130的，80-90的，要不是我去医院拿...,92,2020-11-16 21:01:30,1,10,0,0,10,2020-11-16,2021-01-24,69,2,1,0.222458
2,12329148322,布***咔,不准，比医院用的高了11,12,2019-01-02 14:39:18,1,17,1,1,30,2019-01-02,2021-01-24,753,1,1,0.150000
3,12737339981,j***z,真的垃圾啊，不到一个月，显示屏就出问题了,20,2019-05-07 21:49:36,1,10,0,0,39,2019-05-07,2021-01-24,628,1,1,0.135833
4,14050194816,心***福,测不注意测不准就这玩意还是算了吧，根本,19,2020-04-18 20:29:44,1,23,0,0,66,2020-04-18,2021-01-24,281,0,1,0.282727
